# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  May 20 2022  1:56PM  237764        19    DEX0006840   
1  May 20 2022  1:40PM  237763        10       8045364   
2  May 20 2022  1:36PM  237762        10       8045391   
3  May 20 2022  1:21PM  237761        20       8045388   
4  May 20 2022  1:18PM  237760        10       SO94257   
5  May 20 2022  1:08PM  237759        10  SOIVA0000477   
6  May 20 2022  1:08PM  237759        10  SOIVA0000478   
7  May 20 2022  1:08PM  237759        10  SOIVA0000479   
8  May 20 2022 12:57PM  237758        10    0085694589   
9  May 20 2022 12:57PM  237758        10    0085694588   

                                    Customer ShipmentStatus  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx       Released  
1                           Eywa Pharma Inc.       Released  
2                                    Bio-PRF      Executing  
3                Reliable 1 Laboratories LLC       Released  
4                   Senseonics, Incorporated       Released  
5                       Ivaoes Animal Health      Executing  
6                       Ivaoes Animal Health       Released  
7                       Ivaoes Animal Health       Released  
8                          ISDIN Corporation       Released  
9                          ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
32  237760       Released          1
33  237761       Released          1
34  237762      Executing          1
35  237763       Released          1
36  237764       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
237760                NaN        NaN       1.0
237761                NaN        NaN       1.0
237762                NaN        1.0       NaN
237763                NaN        NaN       1.0
237764                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237693                0.0        0.0       1.0
237694                0.0        0.0       1.0
237698                0.0        3.0       2.0
237700                0.0        0.0       1.0
237704                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237693                  0          0         1
237694                  0          0         1
237698                  0          3         2
237700                  0          0         1
237704                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               237693          0          0         1
1               237694          0          0         1
2               237698          0          3         2
3               237700          0          0         1
4               237704          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               237693                            1
1               237694                            1
2               237698                   3        2
3               237700                            1
4               237704                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   May 20 2022  1:56PM  237764        19   
1   May 20 2022  1:40PM  237763        10   
2   May 20 2022  1:36PM  237762        10   
3   May 20 2022  1:21PM  237761        20   
4   May 20 2022  1:18PM  237760        10   
5   May 20 2022  1:08PM  237759        10   
8   May 20 2022 12:57PM  237758        10   
20  May 20 2022 12:51PM  237757        10   
65  May 20 2022 12:47PM  237756        10   
66  May 20 2022 12:45PM  237755        20   

                                     Customer  
0   Dexcel Pharma Technologies Ltd. DBA WDSrx  
1                            Eywa Pharma Inc.  
2                                     Bio-PRF  
3                 Reliable 1 Laboratories LLC  
4                    Senseonics, Incorporated  
5                        Ivaoes Animal Health  
8                           ISDIN Corporation  
20                          ISDIN Corporation  
65                                Emerginnova  
66                Reliable 1 Laboratories LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  May 20 2022  1:56PM  237764        19   
1  May 20 2022  1:40PM  237763        10   
2  May 20 2022  1:36PM  237762        10   
3  May 20 2022  1:21PM  237761        20   
4  May 20 2022  1:18PM  237760        10   
5  May 20 2022  1:08PM  237759        10   
6  May 20 2022 12:57PM  237758        10   
7  May 20 2022 12:51PM  237757        10   
8  May 20 2022 12:47PM  237756        10   
9  May 20 2022 12:45PM  237755        20   

                                    Customer Completed Executing Released  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx                            1  
1                           Eywa Pharma Inc.                            1  
2                                    Bio-PRF                   1           
3                Reliable 1 Laboratories LLC                            1  
4                   Senseonics, Incorporated                            1  
5                       Ivaoes Animal Health                   1        2  
6                          ISDIN Corporation                           12  
7                          ISDIN Corporation                   4       41  
8                                Emerginnova                            1  
9                Reliable 1 Laboratories LLC                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  May 20 2022  1:56PM  237764        19   
1  May 20 2022  1:40PM  237763        10   
2  May 20 2022  1:36PM  237762        10   
3  May 20 2022  1:21PM  237761        20   
4  May 20 2022  1:18PM  237760        10   
5  May 20 2022  1:08PM  237759        10   
6  May 20 2022 12:57PM  237758        10   
7  May 20 2022 12:51PM  237757        10   
8  May 20 2022 12:47PM  237756        10   
9  May 20 2022 12:45PM  237755        20   

                                    Customer Released Executing Completed  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx        1                      
1                           Eywa Pharma Inc.        1                      
2                                    Bio-PRF                  1            
3                Reliable 1 Laboratories LLC        1                      
4                   Senseonics, Incorporated        1                      
5                       Ivaoes Animal Health        2         1            
6                          ISDIN Corporation       12                      
7                          ISDIN Corporation       41         4            
8                                Emerginnova        1                      
9                Reliable 1 Laboratories LLC        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  May 20 2022  1:56PM  237764        19   
1  May 20 2022  1:40PM  237763        10   
2  May 20 2022  1:36PM  237762        10   
3  May 20 2022  1:21PM  237761        20   
4  May 20 2022  1:18PM  237760        10   

                                    Customer Released Executing Completed  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx        1                      
1                           Eywa Pharma Inc.        1                      
2                                    Bio-PRF                  1            
3                Reliable 1 Laboratories LLC        1                      
4                   Senseonics, Incorporated        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse  \
0  May 20 2022  1:56PM  237764        19   
1  May 20 2022  1:40PM  237763        10   
2  May 20 2022  1:36PM  237762        10   
3  May 20 2022  1:21PM  237761        20   
4  May 20 2022  1:18PM  237760        10   

                                    Customer  Released  Executing  Completed  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx       1.0        NaN        NaN  
1                           Eywa Pharma Inc.       1.0        NaN        NaN  
2                                    Bio-PRF       NaN        1.0        NaN  
3                Reliable 1 Laboratories LLC       1.0        NaN        NaN  
4                   Senseonics, Incorporated       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  May 20 2022  1:56PM  237764        19   
1  May 20 2022  1:40PM  237763        10   
2  May 20 2022  1:36PM  237762        10   
3  May 20 2022  1:21PM  237761        20   
4  May 20 2022  1:18PM  237760        10   

                                    Customer  Released  Executing  Completed  
0  Dexcel Pharma Technologies Ltd. DBA WDSrx       1.0        0.0        0.0  
1                           Eywa Pharma Inc.       1.0        0.0        0.0  
2                                    Bio-PRF       0.0        1.0        0.0  
3                Reliable 1 Laboratories LLC       1.0        0.0        0.0  
4                   Senseonics, Incorporated       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3328334      66.0        6.0        0.0
15          713160       2.0       27.0        7.0
16          237742       1.0        0.0        0.0
18         1664203       7.0        0.0        0.0
19          475462       3.0        3.0        0.0
20          950980      12.0        0.0        0.0
21          237724       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3328334      66.0        6.0        0.0
1        15   713160       2.0       27.0        7.0
2        16   237742       1.0        0.0        0.0
3        18  1664203       7.0        0.0        0.0
4        19   475462       3.0        3.0        0.0
5        20   950980      12.0        0.0        0.0
6        21   237724       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      66.0        6.0        0.0
1        15       2.0       27.0        7.0
2        16       1.0        0.0        0.0
3        18       7.0        0.0        0.0
4        19       3.0        3.0        0.0
5        20      12.0        0.0        0.0
6        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   66.0
1        15  Released    2.0
2        16  Released    1.0
3        18  Released    7.0
4        19  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   18   19    20   21
Status                                         
Completed   0.0   7.0  0.0  0.0  0.0   0.0  0.0
Executing   6.0  27.0  0.0  0.0  3.0   0.0  0.0
Released   66.0   2.0  1.0  7.0  3.0  12.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   18   19    20   21
0          Completed   0.0   7.0  0.0  0.0  0.0   0.0  0.0
1          Executing   6.0  27.0  0.0  0.0  3.0   0.0  0.0
2           Released  66.0   2.0  1.0  7.0  3.0  12.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   18   19    20   21
0  Completed   0.0   7.0  0.0  0.0  0.0   0.0  0.0
1  Executing   6.0  27.0  0.0  0.0  3.0   0.0  0.0
2   Released  66.0   2.0  1.0  7.0  3.0  12.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()